<h3><b> Kaggle: Intermediate Machine Learning </b></h3>

Dummy dataframe and train/valuation segmentation:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("file_name.csv")
y = df."predictor_col_name"      
exp_rows = ["col_name1", "col_name2", ...]
X = df[exp_rows]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

<b> Chapter 2: Missing Values </b>

Approaches to missing values:
1. Drop columns
2. Imputation (Replace with values: mean, median etc...)
3. Imputation with new binary column suggesting missing value

In [ ]:
# Libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 

# Function returning error size of models
def score_dataset(train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(train_X, train_y)
    preds = model.predict(val_X)
    return mean_absolute_error(val_y, preds) # Can replace MAE with other types of error metric

# 1. Dropping columns with missing values
cols_with_missing = [col for col in train_X.columns
                     if train_X[col].isnull().any()]

<b> Chapter 3: Categorical Variables </b>

<b> Chapter 4: Pipelines </b>

<b> Chapter 5: Cross-Validation </b> 

<b> Chapter 6: XGBoost </b>

<b> Chapter 7: Data Leakage </b>